In [3]:
### Initial load
# Import Packages
import pandas as pd

# Load data files
community_df = pd.read_csv("data/community.csv")
news_df = pd.read_csv("data/news.csv")
multilingual_df = pd.read_csv("data/multilingual.csv")

# Test loaded files
community_df.head(5)
news_df.head(5)
multilingual_df.head(5)

,question,answer,url,language,source,section,wrong_answer
0,什么是新型冠状病毒？,新型冠状病毒是一种此前尚未确定的新冠状病毒。导致新型冠状病毒肺炎 | 新冠肺炎2019（CO...,https://chinese.cdc.gov/coronavirus/2019-ncov/...,chinese,CDC,Frequently Asked Questions,使用 CDC 开发的诊断检测，结果为阴性是指在受测者样本中未发现导致 COVID-19 的病...
1,为什么这种疾病被称为 2019 年新型冠状病毒肺炎（COVID-19）？,2020 年 2 月 11 日，世界卫生组织宣布了导致最初于中国武汉发现的 2019 年新型...,https://chinese.cdc.gov/coronavirus/2019-ncov/...,chinese,CDC,Frequently Asked Questions,引发COVID-19的病毒被认为主要以人传人的方式，通过感染患者咳嗽或打喷嚏时产生的呼吸道飞...
2,为什么有人会因为 COVID-19 而指责或躲避某些个人和群体（制造污名）？,美国民众可能对住在或前往COVID-19传播地区的亲友感到担心焦虑。有些人很担心通过这些人感...,https://chinese.cdc.gov/coronavirus/2019-ncov/...,chinese,CDC,Frequently Asked Questions,目前，没有证据表明COVID-19病毒会从宠物的皮肤或皮毛传播给人类。\n关于照顾宠物的最佳...
3,人们如何制止与 COVID-19 相关的污名化？,在注意到污名化正在发生的情况下，人们可以通过提供社会支持来对抗污名化。污名会影响被污名化群体...,https://chinese.cdc.gov/coronavirus/2019-ncov/...,chinese,CDC,Frequently Asked Questions,洗手是保护您与您的家人远离疾病的最佳途径之一。经常用肥皂和水洗手20秒以上，尤其是擤鼻涕、咳...
4,为何某些州的COVID-19病例数量有时与CDC网站上发布的数量有差别？,CDC发布的总病例数量依照各辖区的确认程序得以验证。不同的地方所显示的病例数量的发现和确认程...,https://chinese.cdc.gov/coronavirus/2019-ncov/...,chinese,CDC,Frequently Asked Questions,\n\n与孩子的学校保持联系。\n\n很多学校提供网络课程（虚拟学习）。查看学校布置的作业，...


In [ ]:
### Preprocessing